# Team Fortress 2 Economics

This notebook is a work-in-progress study of the Team Fortress 2 economy. It utilizes the `backpack.tf` [price history API](https://backpack.tf/api/pricehistory). The Python code backending the analysis is my own, but massive credit goes to [Fiskie](http://steamcommunity.com/profiles/76561198012598620/), the `backpack.tf` developer who put together the API that made this possible.

## FAQ

### What is Team Fortress 2?

Team Fortress 2 is a first-person team-based multiplayer shooter video game that was initially released in 2007. The game has recieved continual love and attention from its publisher, Valve, ever since then, which a handful of major content updates released every year and a variety of smaller fixes and additions released throughout. The result is that the game remains viable today, though I personally would say that at this point—going on eight years post-release—its glory days are behind it.

If you are not already familiar with the game, the [Team Fortress 2](https://en.wikipedia.org/wiki/Team_Fortress_2) Wikipedia article is, as always, a good reference point.

### What is the Team Fortress 2 economy? Where did it come from?

Team Fortress 2 was originally a pay-to-play game: at the time of its 2007 release you had to buy the game disc, or download it from the then still new [Steam](https://en.wikipedia.org/wiki/Steam_%28software%29) platform, in order to play it; once you owned the game you owned the game, and Valve neither had nor produced any further incentive for your monetary investment in the game.

Upon release in 2007, TF2 featured a small number of maps, a few stock weapons, and gratuitous cartoon violence—but little else. Now, fast forward to Septmeber 2010. Successive updates have introduced a small armory of new weapons, the item crafting system, hats, and other such nicities, but it's been three years now, and the game is turning into a money sink for
Valve. The Team Fortress 2 economy was their clever solution: a storefront was introduced allowing players to spend real word money on pixilated cosmetics and player items, and, simultaneously and critically, a trading interface. To encourage trading and valuation of goods they turned to the market: a trading interface was created that allowed people to swap items with one another.

The economy has had its ups and downs and bumps ever since then, but one thing is clear: it made Valve money; a lot of money. No one knows exactly how much money is locked up in the Team Fortress 2 economy, but it's [probably in the hundreds of millions of dollars](http://kotaku.com/5869042/analyst-pegs-team-fortress-2-hat-economy-at-50-million).


### Who participates in the economy?

Technically speaking, everyone that is playing the game participates: the game (mostly) distributes the weapons that players can unlock and use for their characters using a time-based drop system, and these items—more specifically the "metal" that they can be "melted down" into—each have a small but non-trivial monetary value.

In reality, the vast majority of trading activities are untaken by a specific subset of "hardcore" players, players who invest incredible amounts of time and money into their activities. Studying these players is not the point of this notebook, but it might be interesting to perform a longitudinal study of trading activity, as I am not aware of one that exists.


### What makes an item valuable?

The short answer: [Hats](https://www.youtube.com/watch?v=HDI2EmYH98I).

The long answer: Price is primarily a function of two things: rarity and stylishness. Truly valuable items—cosmetics, primarily—drop very rarely. Extremely valuable ones are distributed on promotional or store-only grounds, and do not drop at all. And the absolute most valuable items are "unusual" hats which have various particle effects attached to themselves. Similarly, just as real life fashions can drive prices, so too can Team Fortress 2 ones.


### What is the most valuable item?

At the intersection of fashion and rarity is the hat to end all hats, the [Burning Flames Unusual Team Captain](https://backpack.tf/stats/Unusual/Team%20Captain/Tradable/Craftable/13). The exact transaction costs of these items is sketchy, but `backpack.tf` says that at one point they were selling for over over 19,000 dollars.

For a virtual hat.

Think about that.


### What is the largest ever single transaction?

I would love to know myself. The most correct answer might not even be public information.


### How far back does the market data used for this study go?

`backpack.tf` data goes back to late 2012: the initial log for Earbuds goes back to `2012-08-10`, while the initial log for keys goes back to `2012-07-09`. The constraining variable is Metal, the USD price for which was only first logged on `2013-01-07`. Since all prices are passed through Metal on the way to USD, this means that reliable prices can only be established back to (a reasonably extrapolated) `2013-01-01`.

Future work may extend the augment the timeline back beyond this date, this is up in the air - still exploring alternative data sources.

For the moment price data older than `2013-01-01` is removed, and all prices are normalized to `usd` by default. This could change in the final version.


### Who are you? Why are you doing this?

I am an undergraduate student of mathematics in New York City. On Steam I go by [ResMar](https://steamcommunity.com/id/residentmario). I find video game economics to be a fascinating subject and have always wanted to study it in some manner.


### Where can I learn more about this economy?

My own recommended reading list:

* http://scientificgamer.com/an-economy-of-hats/
* http://www.gamefaqs.com/pc/437678-team-fortress-2/faqs/65012
* http://blogs.valvesoftware.com/economics/arbitrage-and-equilibrium-in-the-team-fortress-2-economy/

I welcome suggestions by others.


## Code

The following sequence of code cells constructs an `sqlite3` database at `./tf2_items.db` containing two tables: an `items` table containing definitions for all marketable items in the game; and a a `market_history` table containing the full market price history.

* **To recompile from source**: Run all of the code blocks below. The queries needed to construct the dataset take ~10 minutes to run (more once I start scraping TF2 wiki pages).
* **To use the data immediately**: Usually the above shouldn't be necessary, as `tf2_items.db` is distributed with this repository, so you can merely instantiate the items database (using the `sqlite3 tf2_items.db` shell command for instance) and use it yourself immediately. The only code block you have to run is the [Loader](#Loader-code) block.

### Constructive code

First construct a basic `sqlite3` table of all of the marketable items in Team Fortress 2 using the [backpack.tf pricelist spreadsheet](https://backpack.tf/pricelist/spreadsheet):

In [1]:
import sqlite3
import os
from bs4 import BeautifulSoup
import requests
import urllib


def get_all_items():
    """
    Parses the backpack.tf item spreadsheet in order to retrieve every non-usual item in the game which is on the
    market.
    :return:
    """
    spreadsheet = BeautifulSoup(requests.get("http://backpack.tf/pricelist/spreadsheet").text, 'html.parser')
    item_strings = spreadsheet.find_all("a", {"class": 'qlink'})
    ret = []
    for (item_string, num) in zip(item_strings, range(len(item_strings))):
        item_string = str(item_string)
        repr = (
            urllib.request.unquote(item_string.split("/")[3]),
            urllib.request.unquote(item_string.split("/")[2]),
            item_string.split("/")[4],
            item_string.split("/")[5][:item_string.split("/")[5].find('"')],
            num
        )
        ret.append(repr)
    return ret
 
if os.path.isfile('tf2_items.db'):
    print("Connecting to database...")
else:
    print("Initializing database...")
conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
print("Resetting tables...")
c.execute('''DROP TABLE if exists items''')
conn.commit()
c.execute('''CREATE TABLE items
  (name text,quality text, tradable integer, craftable integer, price_index integer PRIMARY KEY)
''')
conn.commit()
print("Retrieving marketable items list...")
items_in = get_all_items()
print("Storing marketable items in database...")
c.executemany('INSERT INTO items VALUES (?,?,?,?,?)', items_in)
conn.commit()
conn.close()
print("Done!")

Connecting to database...
Resetting tables...
Retrieving marketable items list...
Storing marketable items in database...
Done!


Next construct a market information schema. This turned out to be a very verbose task. The `Item` class (below) handles all market value conversion, storing high and low market values in an intermediary `.history pandas DataFrame` object attribute.

In [2]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import requests
import json
import os
from bs4 import BeautifulSoup
import urllib
import time
import arrow


class Item:
    """
    Each TF2 item is represented as a `pandas` `DataFrame` object demarcating its historical cost table.
    """

    name = ""
    quality = ""
    craftability = ""
    tradability = ""
    history = DataFrame()

    def __repr__(self):
        return str(self.history)

    def __init__(self, item, **kwargs):
        """
        Every Item has an associated filename, generated by `str_repr` below. If the associated file is present in
        the folder initialization loads the data from there. If it is not initialization makes an API query,
        loads the results, and stores the file---for future reference.
        """
        self.name = item
        if item == "Mann Co. Supply Crate Key":
            key = _key_prices()
            self.history = key.history
        elif item == "Refined Metal":
            metal = _metal_prices()
            self.history = metal.history
        else:
            str_repr = ""
            if 'quality' not in kwargs.keys():
                str_repr += "unique "
                self.quality = "unique"
            else:
                str_repr += kwargs['quality'].lower() + " "
                self.quality = kwargs['quality'].lower()
            if 'craftable' not in kwargs.keys() or kwargs['craftable'] == 1 or kwargs['craftable'] == 'Craftable':
                str_repr += "craftable "
                self.craftability = "craftable"
            else:
                str_repr += "non-craftable "
                self.craftability = "non-craftable"
            if 'tradable' not in kwargs.keys() or kwargs['tradable'] == 1 or kwargs['tradable'] == 'Tradable':
                str_repr += "tradable "
                self.tradability = "tradable"
            elif kwargs['tradable'] == 0:
                str_repr += "non-tradable "
                self.tradability = "non-tradable"
            if "{0}{1}.csv".format(str_repr, item.lower()) in [f for f in os.listdir('.') if os.path.isfile(f)]:
                self.read_csv("{0}{1}.csv".format(str_repr, item.lower()))
            else:
                if len(item) > 0:
                    # Make the API call.
                    params = {"key": _get_key(), "item": item}
                    params.update(kwargs)
                    data = json.loads(requests.get("http://backpack.tf/api/IGetPriceHistory/v1/",
                                                   params=params).text)['response']['history']
                    # Parse it into a `pandas` `DataFrame`.
                    frame = DataFrame(data,
                                      index=[np.datetime64(str(arrow.get(t['timestamp']).format('YYYY-MM-DD'))) for t in
                                             data],
                                      columns=["currency", "value", "value_high"])
                    # Throw out earlier dates, for the moment. `2013-01-01` is the starting point for all knowledge.
                    frame = frame[pd.to_datetime('2013-01-01'):]
                    # Convert currency.
                    metal = _metal_prices()
                    key = _key_prices()
                    # Throw out "hat" currencied values. Weird stopgap that they used briefly (hopefully).
                    frame = frame[frame['currency'] != 'hat']
                    for i in range(0, len(frame.index)):
                        data = frame.iloc[i]
                        date = data.name
                        currency = data['currency']
                        conversion_rate = _value_at(currency, date, key, metal)
                        frame.iat[i, 0] = 'usd'
                        # FAILURE POINT: Item instantiation fails here.
                        # print(conversion_rate[0])
                        # print(frame.iat[i, 1])
                        frame.iat[i, 1] *= conversion_rate[0]
                        frame.iat[i, 2] *= conversion_rate[1]
                    self.history = frame
                    # self.to_csv("{0}{1}.csv".format(str_repr, item.lower()))

    def to_csv(self, filename):
        """
        Pass-through wrapper that saves the `Item` in CSV.
        :param filename: The filename at which to save the `Item`.
        """
        self.history.to_csv(filename)

    def read_csv(self, filename):
        """
        Pass-through wrapper that loads the `Item` in CSV.
        :param filename: The filename from which to load the `Item`.
        """
        self.history = self.history.from_csv(filename)


def _value_at(currency, date, key, metal):
    """
    Converts one currency into USD based on a date. To speed up the operation key and metal are passed to this
    internal method so that the required table lookups only need happen once.
    :param currency: String representing the currency being converted into USD, one of ["usd", "metal", "keys",
    or "hat"]. That last one...I just throw hat out in the in-definition smoothing step.
    :param date: The date of the conversion.
    :param key: A key `DataFrame` history object.
    :param metal: A metal `DataFrame` history object.
    :return: The value of the given currency at the given date in USD.
    """
    f_date = np.datetime64(str(arrow.get(date).format('YYYY-MM-DD')))
    # USD-to-USD conversion is a multiple of 1.
    if currency == "usd":
        return 1, 1
    # Convert metal prices based on historical data.
    elif currency == "metal":
        value_entry = metal.history[:f_date].tail(1)
        return value_entry['value'][0], value_entry['value_high'][0]
    # Convert key prices based on historical data.
    elif currency == "keys":
        value_entry = key.history[:f_date].tail(1)
        return value_entry['value'][0], value_entry['value_high'][0]


def _get_key(filename='backpack_tf_account_credentials.json'):
    if filename in [f for f in os.listdir('.') if os.path.isfile(f)]:
        return json.load(open(filename))['credentials']['token']
    else:
        raise IOError(
                'This API requires a backpack.tf credentials token to work. Did you forget to generate one? For more '
                'information refer to:\n\nhttps://backpack.tf/api/pricehistory')


def _metal_prices(filename='unique craftable tradable refined metal.csv'):
    """
    Retrieves the metal price table. Since this operation is needed whenever a conversion is made the data is saved
    locally as a CSV and extracted from there.
    :param filename: The filename at which metal prices are saved.
    :return: The `DataFrame` price history object.
    """
    if filename in [f for f in os.listdir('.') if os.path.isfile(f)]:
        ret = Item("")
        ret.read_csv(filename)
        return ret
    else:
        params = {"key": _get_key(), "item": "Refined Metal"}
        data = json.loads(requests.get("http://backpack.tf/api/IGetPriceHistory/v1/",
                                       params=params).text)['response']['history']
        # Parse it into a `pandas` `DataFrame`.
        frame = DataFrame(data,
                          index=[np.datetime64(str(arrow.get(t['timestamp']).format('YYYY-MM-DD'))) for t in
                                 data],
                          columns=["currency", "value", "value_high"])
        # Extrapolate Refined Metal price back to `2013-01-01`.
        frame.loc[pd.to_datetime('2013-01-01')] = ['usd', 0.40, 0.40]
        frame = frame.sort_index()
        metal = Item("")
        metal.history = frame
        metal.craftability = "craftable"
        metal.tradability = "tradable"
        metal.to_csv(filename)
        return metal


def _key_prices(filename='unique craftable tradable mann co. supply crate key.csv'):
    """
    Retrieves the key price table.
    :param filename: The filename at which key prices are saved.
    :return: The `DataFrame` price history object.
    """
    if filename in [f for f in os.listdir('.') if os.path.isfile(f)]:
        ret = Item("")
        ret.read_csv(filename)
        return ret
    else:
        params = {"key": _get_key(), "item": "Mann Co. Supply Crate Key"}
        data = json.loads(requests.get("http://backpack.tf/api/IGetPriceHistory/v1/",
                                       params=params).text)['response']['history']
        # Parse it into a `pandas` `DataFrame`.
        frame = DataFrame(data,
                          index=[np.datetime64(str(arrow.get(t['timestamp']).format('YYYY-MM-DD'))) for t in
                                 data],
                          columns=["currency", "value", "value_high"])
        frame = frame[pd.to_datetime('2013-01-01'):]
        metal = _metal_prices()
        for i in range(0, len(frame.index)):
            data = frame.iloc[i]
            date = data.name
            frame = frame[pd.to_datetime('2013-01-01'):]
            currency = data['currency']
            conversion_rate = _value_at(currency, date, None, metal)
            frame.iat[i, 0] = 'usd'
            frame.iat[i, 1] *= conversion_rate[0]
            frame.iat[i, 2] *= conversion_rate[1]
        key = Item("")
        key.history = frame
        key.craftability = "craftable"
        key.tradability = "tradable"
        key.to_csv(filename=filename)
        return key


def get_all_items():
    """
    Returns a list of all non-unusual items in the game.
    :return:
    """
    spreadsheet = BeautifulSoup(requests.get("http://backpack.tf/pricelist/spreadsheet").text, 'html.parser')
    item_strings = spreadsheet.find_all("a", {"class": 'qlink'})
    # return item_strings
    # return [urllib.request.unquote(str(item_string)).split("/")[3] for item_string in item_strings]
    ret = []
    for item_string in item_strings:
        item_string = str(item_string)
        string_repr = "{0} {1} {2} {3}".format(item_string.split("/")[2],
                                               item_string.split("/")[4],
                                               item_string.split("/")[5][:item_string.split("/")[5].find('"')],
                                               urllib.request.unquote(item_string.split("/")[3]))
        # string_repr = urllib.request.unquote(str(item_string).split("/")[3])
        ret.append(string_repr)
    return ret

The `Item` class allows us to easily table individual items' market values. For example, these are the most recently known values for the price of a [pair of earbuds](https://wiki.teamfortress.com/wiki/Earbuds):

In [3]:
Item("Earbuds").history.tail(5)

,currency,value,value_high
2015-12-29,usd,5.61492,6.19866
2016-01-18,usd,5.61492,6.88740
2016-02-23,usd,5.19900,6.88740
2016-03-03,usd,5.19900,5.73950
2016-03-05,usd,4.99104,5.50992


Now we build a `pandas <-> sqlite3` bridge so that we can go there and back again. First we add a `market_history` store to the database.

In [23]:
conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
print("Resetting table...")
c.execute('''DROP TABLE if exists market_history''')
conn.commit()
print("Keying new table...")
c.execute('''CREATE TABLE market_history
  (price_index integer, date text, price_low real, price_high real)
''')
c.execute(''' CREATE INDEX item_index ON market_history (price_index)''')
conn.commit()
conn.commit()
conn.close()
print("Done!")

Resetting table...
Keying new table...
Done!


Make sure the currency conversions (stored in a `csv` file as an intermediary) are ready.

In [6]:
currencies = [Item("Refined Metal"), Item("Mann Co. Supply Crate Key")]

Finally the motherlode operation: the mass market insertion.

In [22]:
def insert(item, price_index, cursor):
    dat = [(price_index, str(row[0]), str(row[1][1]), str(row[1][2])) for row in item.history.iterrows()]
    for tup in dat:
        cursor.execute('''INSERT into market_history VALUES (?,?,?,?)''', tup)

In [27]:
from tqdm import tqdm


conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
dat = c.execute('''SELECT * from items''')
conn.commit()
for item in tqdm(list(dat)):
    try:
        dat = Item(item[0], rarity=item[1], tradable=item[2], craftable=item[3])
        insert(dat, item[4], c)
        conn.commit()
    except:
        print("We failed trying to instantialize Item({0}, {1}, {2}, {3})".format(item[0], item[1], item[2], item[3]))
#         break
        continue
conn.close()

All ready!

In [34]:
conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
dat = c.execute('''SELECT price_index from market_history''')
print(len(list(dat)))
conn.close()

44827


That's a lot of history!

Next use a web scrape of the [Team Fortress 2 Wiki](https://wiki.teamfortress.com/) to associate each of the items in this items bank with categorical information about, amongst other things, the date that it was added to the game. Most of this information is read off of the categories; date of introduction is read off of the first entry in the item's **Update history** section ([example](https://wiki.teamfortress.com/wiki/Bill's_Hat)).

This data is taken from an [API call](https://www.mediawiki.org/wiki/API:FAQ#get_the_content_of_a_page_.28HTML.29.3F) for the [wikitext](https://en.wikipedia.org/wiki/Wiki_markup) of the page.

First grab the item list.

In [220]:
conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
items_list = [tup[0] for tup in list(c.execute('''SELECT name from items'''))]
conn.close()

Next we actually hit the Team Fortress 2 wiki to retrieve the data that we want. Since this query has to go through 3579 wiki pages, it takes about 20 minutes to run in full. A `DataFrame` serves as an intermediary.

In [221]:
import mwparserfromhell
import arrow
import re


def isolate_wikilink(string):
    """
    Helper function which uses regex to handle extracting a link from a wikilink string.
    For example, `[[Apple]] -> Apple`, and `[[Class|All classes]]` -> `All classes`.
    Note: group() will blow up in the case that the string is empty. Thus this method includes exception
    handling for this specific case, in which case it will hand off the standard `None` response.
    `Burned Banana Peel` is an example of a page for which this error handling is necessary.
    """
    if "|" in string:
        string = '[[' + string[string.rfind("|") + 1:]
    ret = re.search('(?<=\[\[)[^\]]*(?=[\]])', string)
    try:
        return ret.group()
    except AttributeError:
        return None


intro_dates = []
classes = []
slots = []
log = ""


for item in tqdm(items_list):
    intro_date = used_by = slot_row = slot = None
    page = "https://wiki.teamfortress.com/w/index.php?action=raw&title={0}".format(item)
    wikicode = mwparserfromhell.parse(requests.get(page).text)
    templates = wikicode.filter_templates()
    # The first instance of a {{Patch name}} template on the page will point to the date of introduction of the item.
    try:
        intro_patch = next(template for template in templates if 'atch name' in template.name)
    # No infobox was found! In this case don't do anything else, but print an error-check message.
    # This means that this particular item does not have its own TF2-Wiki page.
    # For example, this is true of all of the paints, which all point to the `Paint` article.
    except StopIteration:
        log += "No patch was found on page '{0}' for item '{1}'.\n".format(page, item)
        intro_dates.append(intro_date)
        classes.append(used_by)
        slots.append(slot)
        continue
    # If this raises a StopIteration then no patch template was found! This is bad.
    intro_date = arrow.Arrow(int(str(intro_patch.get(3).value)),
                             int(str(intro_patch.get(1).value)),
                             int(str(intro_patch.get(2).value)))
    # Now parse the infobox.
    try:
        infobox = next(template for template in templates if 'tem infobox' in template.name)
    except StopIteration:
        # No infobox was found! In this case don't do anything else, but print an error-check message.
        log += "No infobox was found on page {0} for item {1}\n.".format(page, item)
        intro_dates.append(intro_date)
        classes.append(used_by)
        slots.append(slot)
        continue
    # If the item has classes associated with it, extract those.
    try:
        used_by_row = next(row for row in infobox.params if 'used-by' in row)
        used_by = []
        for class_user in used_by_row.split(","):
            used_by.append(isolate_wikilink(str(class_user)))
    except StopIteration:
        pass
    # If the item has a slot associated with it, extract that.
    try:
        slot_row = next(row for row in infobox.params if 'type' in row)
        slot = re.search('(?<=[=])[\S]*', str(slot_row).replace(" ", ""))
        try:
            slot = slot.group()
        # In a handful of cases a 'type = ' row is defined for the infobox template, but not filled in.
        # In that case we skip along, writing a "None" for this column, as intended.
        except AttributeError:
            pass
    except StopIteration:
        pass
    intro_dates.append(intro_date)
    classes.append(used_by)
    slots.append(slot)

print(log)
    
# Use a DataFrame to verify that the operation ran correctly.
df = DataFrame({
            'Items': items_list,
            'Introduction date': intro_dates,
            'Classes used by': classes,
            'Slot': slots
        })
df.head(10)

No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=A Color Similar to Slate' for item 'A Color Similar to Slate'.
No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=A Deep Commitment to Purple' for item 'A Deep Commitment to Purple'.
No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=A Distinctive Lack of Hue' for item 'A Distinctive Lack of Hue'.
No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=A Distinctive Lack of Hue' for item 'A Distinctive Lack of Hue'.
No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=A Mann's Mint' for item 'A Mann's Mint'.
No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=After Eight' for item 'After Eight'.
No patch was found on page 'https://wiki.teamfortress.com/w/index.php?action=raw&title=Aged Moustache Grey' for item 'Aged Moustache Grey'

,Classes used by,Introduction date,Items,Slot
0,[Medic],2013-10-10T00:00:00+00:00,A Brush with Death,cosmetic
1,None,None,A Color Similar to Slate,None
2,None,None,A Deep Commitment to Purple,None
3,None,None,A Distinctive Lack of Hue,None
4,None,None,A Distinctive Lack of Hue,None
5,[Spy],2015-12-17T00:00:00+00:00,A Hat to Kill For,cosmetic
6,[Spy],2015-12-17T00:00:00+00:00,A Hat to Kill For,cosmetic
7,[Pyro],2015-10-06T00:00:00+00:00,A Head Full of Hot Air,cosmetic
8,[Pyro],2015-10-06T00:00:00+00:00,A Head Full of Hot Air,cosmetic
9,None,None,A Mann's Mint,None


There are a number of inconsistencies that now need manual fixing. We can do this either before (using `pandas`) or after (using `SQL`) writing the data to the database. I chose to do so before because even though in this specific case we're rolling a database, in most instances as a data analyst one should be manipulating the output, not the input - your corrections ought to be just that, corrections, not self-instantiated ground truths!

Errors:

* None of the `Paint` type items have data on anything besides their name: this is because the TF2-Wiki page for the paint buckets is simply [Paint Cans](https://wiki.teamfortress.com/wiki/Paint_Cans), covering all of them at once. The same is true for noise makers, botkiller weapons, Australium weapons, festive weapons, festive Australium weapons (at what point have you jumped the shark?), slot tokens, time-limited keys, crates, strange parts, strange filters, taunts, holloween masks, and spells.

  Some of these items were released all at once, and are easy to manually correct, but some were released slowly over time, and are very difficult to manually correct.


* A semantic distinction needs to be made between `All` and `All classes` columns entries in `Classes used by`. `All` indicates that the item **is not used in-game at all** - rather, it is a meta-item, something like a [Backpack Expander](https://wiki.teamfortress.com/wiki/Backpack_Expander) which serves its purpose in the backpack setting, not the playable character one. Some of the infoboxes handle this with an `All` entry; some handle this by not defining any classes at all, either leaving the template field blank or not including it all.

  `All class` items on the contrary **are** items which are usable in-game, but which have the property that they can be used by any class. [Bill's Hat](https://wiki.teamfortress.com/wiki/Bill's_Hat) or the ubiquitious [Earbuds](https://wiki.teamfortress.com/wiki/Earbuds) are examples.

These errors are very hard to correct. Unfortunately I think they invalidate the TF2-Wiki scraping approach, so let's try an alternative tack.

Now plug this information into the database. To do this:

1. Normalize the `DataFrame`. Specifically:
  * Convert the list of classes stored in `Classes used by` to a set of single-entry binary fields.
  * Convert the `datetimes` stored as `Arrow` objects to dates readable by `sqlite`. Note that `SQL` defines no special classes for dates: handling these is defined on the side of the DBMS, so in this case, by `sqlite`. ¯\\_(ツ)_/¯
2. Write the `DataFrame` to the `sqlite` database. To accomplish this I use `DataFrame` `to_sql()`. Now that we've normalized everything, it should map fine.
3. Expand the old `item` schema to include the additional data. The easiest way to do this is to `JOIN` the old table and the new one. 

In [184]:
# While we're working on the query, let's define and use a seperate table.

conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
c.execute('''DROP TABLE if exists items_copy''')
c.execute('''CREATE TABLE items_copy AS 
  SELECT *
  FROM items;
''')
# c.execute('''CREATE TABLE items_copy
#   (name text, quality text, tradable integer, craftable integer, price_index integer PRIMARY KEY)
# ''')
# c.execute('''INSERT INTO items_copy SELECT * FROM items;''')
conn.close()

In [185]:
conn = sqlite3.connect('tf2_items.db')
c = conn.cursor()
items_list = [tup for tup in list(c.execute('''SELECT * FROM items_copy LIMIT 5;'''))]
# c.executemany('INSERT INTO items_copy VALUES (?,?,?,?,?)', items_in)
conn.close()
items_list

[('A Brush with Death', 'Unique', 'Tradable', 'Craftable', 0),
 ('A Color Similar to Slate', 'Unique', 'Tradable', 'Craftable', 1),
 ('A Deep Commitment to Purple', 'Unique', 'Tradable', 'Craftable', 2),
 ('A Distinctive Lack of Hue', 'Unique', 'Tradable', 'Craftable', 3),
 ('A Distinctive Lack of Hue', 'Unique', 'Tradable', 'Non-Craftable', 4)]

In [206]:
df2 = df.copy()

In [207]:
df2

,Classes used by,Introduction date,Items,Slot
0,[Medic],2013-10-10T00:00:00+00:00,A Brush with Death,cosmetic
1,None,None,A Color Similar to Slate,None
2,None,None,A Deep Commitment to Purple,None
3,None,None,A Distinctive Lack of Hue,None
4,None,None,A Distinctive Lack of Hue,None
5,[Spy],2015-12-17T00:00:00+00:00,A Hat to Kill For,cosmetic
6,[Spy],2015-12-17T00:00:00+00:00,A Hat to Kill For,cosmetic
7,[Pyro],2015-10-06T00:00:00+00:00,A Head Full of Hot Air,cosmetic
8,[Pyro],2015-10-06T00:00:00+00:00,A Head Full of Hot Air,cosmetic
9,None,None,A Mann's Mint,None


In [203]:
conn = sqlite3.connect('tf2_items.db')
df.to_sql('items_wiki_data', conn)

/Users/Honors/anaconda/envs/tf2-economy/lib/python3.5/site-packages/pandas/core/generic.py:1160: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


InterfaceError: Error binding parameter 1 - probably unsupported type.

In [201]:
for row in df.iterrows():
    tup = (row[1][2], row[1][0], row[1][1], row[0])
    print(tup)

('A Brush with Death', ['Medic'], <Arrow [2013-10-10T00:00:00+00:00]>, 0)
('A Color Similar to Slate', None, None, 1)
('A Deep Commitment to Purple', None, None, 2)
('A Distinctive Lack of Hue', None, None, 3)
('A Distinctive Lack of Hue', None, None, 4)
('A Hat to Kill For', ['Spy'], <Arrow [2015-12-17T00:00:00+00:00]>, 5)
('A Hat to Kill For', ['Spy'], <Arrow [2015-12-17T00:00:00+00:00]>, 6)
('A Head Full of Hot Air', ['Pyro'], <Arrow [2015-10-06T00:00:00+00:00]>, 7)
('A Head Full of Hot Air', ['Pyro'], <Arrow [2015-10-06T00:00:00+00:00]>, 8)
("A Mann's Mint", None, None, 9)
('A Random Duck Token Gift', ['All'], <Arrow [2014-12-08T00:00:00+00:00]>, 10)
('A Random End of the Line Key Gift', ['All'], <Arrow [2014-12-08T00:00:00+00:00]>, 11)
('A Random Robo Community Crate Key Gift', ['All'], <Arrow [2013-05-17T00:00:00+00:00]>, 12)
('A Random Summer Cooler Key Gift', ['All'], <Arrow [2013-07-10T00:00:00+00:00]>, 13)
('A Rather Festive Tree', ['All classes'], <Arrow [2010-12-17T00:00:00

### Loader code

The block below creates a connection to the database without recreating everything, as above.

In [ ]:
# STUFF GOES HERE!

## Market Summary

Before we dive into the particulars of value distributions in Team Fortress 2, let's take a look at the overall economy at scale.

Originally Team Fortress 2 came with a handful of default weapons of "Normal" (or "Stock") quality assigned to each of the playable classes. Today items can have any one of twelve different [item qualities](https://wiki.teamfortress.com/wiki/Item_quality). Though the underlying [texture map](https://en.wikipedia.org/wiki/Texture_mapping) is the same, in market terms items of differing qualities are all distinct from one another in desirability, rarity, and value. The first question we ask is:

### How many distinct items are there?

In [ ]:
unique_count = requests.get("https://tf2stats.net/all_items/").text.count('<div class="item tooltip round5">')
unique_count

This is cerainly a far cry from the 27 items (one primary weapon, one secondary weapon, and one melee weapon for each of the nine playable classes) that existed at the time of the game's release!


**How many distinct tradable items are there?**

An item can have one of a number of different "[qualities](https://wiki.teamfortress.com/wiki/Item_quality)". The quality of an item can have a very large effect on the item's value.

Unfortunately asking what the total number of distinct items on the market is non-trivial because of the way that the "unusual" quality items are distributed. We can nevertheless easily get a count of all items of non-unusual quality.

Note that most of the analysis in the remainder of this notebook uses non-unusual items.

**How many distinct non-unusual tradable items are there?**

In [ ]:
non_unusual_count = requests.get("http://backpack.tf/pricelist/spreadsheet").text.count("title=")
non_unusual_count

**How many different versions does an item have, on average?**

In [ ]:
non_unusual_count / unique_count

**What is the distribution of the values of non-unusual items?**

Prices are basically determined by the laws of supply (how many are available) and demand (how badly the item is desired). The relative levels of these determine the item's price.

**How strongly does price correlate with numerical availability?**

"Desirability" is a characteristic intrinsic to the players that should be captured by the market price. Let's measure how strongly correlated it is with various metrics.

**How much more than average does the highest-value item in a class of items usually sell for?**

Let's look at the economic history of, say, a hat released in a crate. At first it's value will be precariously high, because it's extremely new, fashionable, and no one has one yet. Within a few days of crate-opening, however, its price will start to plummet, driven down by the large number of people that think as you did, and opened crates and got hats of their own, until it bottoms out a week to two weeks after release. Once the crate is taken off the item drops, however, the hat will slowly start to rebound in value again, until stabalizing at a slowly decaying value once it starts appearing in significant numbers in item drops and crafts.

Now let's talk about bubbles. TF2 naturally feeds bubbles. An item's value is rising, and people take notice. They think "well, if I buy it now and sell it later, I'll make money", so they do that - in bulk. The decreased number on sale and increased demand combine the balloon the price to an artificially high value, up until the point where people realize, wait, what are we doing; then the price falls dramatically: the bubble pops. It happens in the real world, and it happens even more often in the layman-covered TF2 economy (real stock traders avoid bubbles). In the past this has happened to the Max Head (from 4 to 2 buds) and the Genuine Maul (from a bud to 7 to 8 keys), for instance, along with a number of other, generally more minor, items.


There are a lot of questions worth asking of this data.

Let's call these our "item classes": unique hats, genuine hats, vintage hats, unusual hats, paint buckets, currencies and psuedo-currencies, action items and taunts, etc...

* How has average item price by category changed over time?
* What does the average hat value curve look like, from the immediate post-release hype to the years that follow?
* Meta: what is the average value percentage adjustment in a `backpack.tf` price?
* How quickly does the value of something from a crate drop?
* etc. etc. etc. etc.

So...first...need to ball all of the API data up into individual CSV files, split up by classes. Since generating a list of "things" in a category is non-trivial to start out let's do some summary statistics on a concrete but interesting subset of the data?